In [1]:
%matplotlib inline
import matplotlib as mpl
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import minmax_scale
from sklearn.decomposition import PCA


In [2]:
#carrega o dataset
dados = pd.read_csv('/home/erasmor/Área de Trabalho/CSE-CIC-IDS2018/todos_apenas_baixa_representatividade.csv',sep=",",encoding = 'utf-8',  header=0,na_values='.')

#remove valores infinitos
dados.replace(-np.Inf, np.nan)

#substitui valores NaN
dados.fillna(dados.mean())


,Dst Port,Protocol,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,80,17,119893441,309629,0,9908128,0,32,32,32.000000,...,8,0.0,0.0,0,0,0.0,0.0,0,0,DDOS attack-LOIC-UDP
1,80,17,119745803,259444,0,8302208,0,32,32,32.000000,...,8,0.0,0.0,0,0,0.0,0.0,0,0,DDOS attack-LOIC-UDP
2,80,17,119965025,248800,0,7961600,0,32,32,32.000000,...,8,0.0,0.0,0,0,0.0,0.0,0,0,DDOS attack-LOIC-UDP
3,80,17,119999991,272337,0,8714784,0,32,32,32.000000,...,8,0.0,0.0,0,0,0.0,0.0,0,0,DDOS attack-LOIC-UDP
4,80,17,119820553,219539,0,7025248,0,32,32,32.000000,...,8,0.0,0.0,0,0,0.0,0.0,0,0,DDOS attack-LOIC-UDP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3468,3389,6,3868068,14,8,1431,1727,725,0,102.214286,...,20,0.0,0.0,0,0,0.0,0.0,0,0,Benign
3469,3389,6,3996691,15,8,1434,1727,725,0,95.600000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,Benign
3470,3389,6,3945545,14,8,1451,1727,741,0,103.642857,...,20,0.0,0.0,0,0,0.0,0.0,0,0,Benign
3471,3389,6,3972090,14,8,1432,1727,725,0,102.285714,...,20,0.0,0.0,0,0,0.0,0.0,0,0,Benign


In [3]:
#mostra como está a base de dados
dados.head()

,Dst Port,Protocol,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,80,17,119893441,309629,0,9908128,0,32,32,32.0,...,8,0.0,0.0,0,0,0.0,0.0,0,0,DDOS attack-LOIC-UDP
1,80,17,119745803,259444,0,8302208,0,32,32,32.0,...,8,0.0,0.0,0,0,0.0,0.0,0,0,DDOS attack-LOIC-UDP
2,80,17,119965025,248800,0,7961600,0,32,32,32.0,...,8,0.0,0.0,0,0,0.0,0.0,0,0,DDOS attack-LOIC-UDP
3,80,17,119999991,272337,0,8714784,0,32,32,32.0,...,8,0.0,0.0,0,0,0.0,0.0,0,0,DDOS attack-LOIC-UDP
4,80,17,119820553,219539,0,7025248,0,32,32,32.0,...,8,0.0,0.0,0,0,0.0,0.0,0,0,DDOS attack-LOIC-UDP


In [4]:
# verifica quantas instâncias (linhas) e quantos atributos (colunas) a base de dados contém
print("numero de linhas e colunas: ",dados.shape)

numero de linhas e colunas:  (3473, 79)


In [5]:
#visualizar distribições por classes contidas no csv - informar nome da classe alvo
print(dados.groupby('Label').size())

Label
Benign                   815
Brute Force -Web         611
Brute Force -XSS         230
DDOS attack-LOIC-UDP    1730
SQL Injection             87
dtype: int64


In [6]:
#X_raw_normalize = MinMaxScaler(X_raw_normalize.reshape(0, 1)).reshape(len(X_raw_normalize))
#X_raw_normalizetd2 = (X_raw_normalize - X_raw_normalize.min(axis=0)) / (X_raw_normalize.max(axis=0) - X_raw_normalize.min(axis=0))
# Obtendo os nomes das colunas do DataFrame como uma lista.
cols = list(dados.columns)
# colunas que nao serao normalizadas
cols.remove('Label')

# Copiando os dados e aplicando a normalizacao por reescala nas colunas do DataFrame que contem
# valores continuos. Por padrao, o metodo minmax_scale reescala com min=0 e max=1.
dados = dados[~dados.isin([np.nan, np.inf, -np.inf]).any(1)]
dados[cols] = dados[cols].apply(minmax_scale)


In [7]:
#define as colunas de atributos e a coluna da classe (de 0 a 72 são atributos e após a 72 é a classe)
# "X_raw" é features/atributos e "y_raw" é target/classe ==> As duas formas abaixo dão certo.
#array = dataset.values
#X_raw = array[:,0:72]
#y_raw = array[:,72]
X_raw = dados.iloc[:, :-1].values # atributos
y_raw = dados.iloc[:, 78].values # classe de ataques
X_raw = np.nan_to_num(X_raw.astype(np.float32))


In [8]:
#transformar a variável Y com valores categóricos das classses de ataques em valores:
labelencoder_y = LabelEncoder()
y_raw = labelencoder_y.fit_transform(y_raw)


In [9]:
def expected_error_reduction_function(X_raw, y_raw, idx_data, idx_dobra, TRAIN, TEST, classifier, init_size):
    
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.naive_bayes import GaussianNB
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.svm import LinearSVC
    from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report, accuracy_score, f1_score
    import functools
    from IPython.display import clear_output
    from copy import deepcopy
    
    #define nome de arquivos para salvar
    indica_pool=str(idx_dobra)
    arquivo_performance_knn = ("expected_error_reduction_performance_knn_dobra_"+indica_pool+".csv")
    arquivo_performance_rf = ("expected_error_reduction_performance_rf_dobra_"+indica_pool+".csv")
    arquivo_performance_nb = ("expected_error_reduction_performance_nb_dobra_"+indica_pool+".csv")
    arquivo_performance_tree = ("expected_error_reduction_performance_tree_dobra_"+indica_pool+".csv")
    arquivo_performance_mlp = ("expected_error_reduction_performance_mlp_dobra_"+indica_pool+".csv")
    
    # recupera as amostras de treino iniciais - a extratificação realizada só serve para tal finalidade.
    # No caso força a buscar pelo menos uma amostras de cada rótulo disponível (train_size= len(np.unique(y_raw)).
    # Realizar a busca aleatoriamente não garante iniciar com uma instância de cada classe.
    X_train, X_test, y_train, y_test = train_test_split(X_raw[idx_data[idx_dobra][TRAIN]], y_raw[idx_data[idx_dobra][TRAIN]], train_size= len(np.unique(y_raw)) + init_size, stratify = y_raw[idx_data[idx_dobra][TRAIN]])
    #print("tamanho de X_train inicial: ",X_train.shape)
    amostras_iniciais=X_train.shape
    n_amostras_iniciais = int(amostras_iniciais[0])
    #define array de performance
    performance_history = []
    # copia para uso no aprendizado inicial
    inicial = deepcopy(X_train)
    #instanciando classificadores de aprendizado ativo
    if classifier == "knn":
        learner = ActiveLearner(estimator=KNeighborsClassifier(n_neighbors=3),X_training=X_train, y_training=y_train)
        arquivo_performance = open("expected_error_reduction_performance_knn_dobra_"+indica_pool+".txt","a")
        arquivo_history = ("expected_error_reduction_history_knn_dobra_"+indica_pool+".csv")
        titulo_grafico = 'Incremental classification accuracy KNN'
        
    if classifier == "rf":
        learner = ActiveLearner(estimator=RandomForestClassifier(max_depth=2),X_training=X_train, y_training=y_train)
        arquivo_performance = open("expected_error_reduction_performance_rf_dobra_"+indica_pool+".txt","a")
        arquivo_history = ("expected_error_reduction_history_rf_dobra_"+indica_pool+".csv")
        titulo_grafico = 'Incremental classification accuracy RF'
        
    if classifier == "nb":
        learner = ActiveLearner(estimator=GaussianNB(),X_training=X_train, y_training=y_train)
        arquivo_performance = open("expected_error_reduction_performance_nb_dobra_"+indica_pool+".txt","a")
        arquivo_history = ("expected_error_reduction_history_nb_dobra_"+indica_pool+".csv")
        titulo_grafico = 'Incremental classification accuracy NB'
        
    if classifier == "tree":
        learner = ActiveLearner(estimator=DecisionTreeClassifier(),X_training=X_train, y_training=y_train)
        arquivo_performance = open("expected_error_reduction_performance_tree_dobra_"+indica_pool+".txt","a")
        arquivo_history = ("expected_error_reduction_history_tree_dobra_"+indica_pool+".csv")
        titulo_grafico = 'Incremental classification accuracy TREE'

        
    
    #verifica a performance inicial
    unqueried_score = learner.score(X_train, y_train)
    performance_history.append(unqueried_score)
    
    # recupera amostras de teste de acordo com a dobra em uso
    X_test, y_test = X_raw[idx_data[idx_dobra][TEST]], y_raw[idx_data[idx_dobra][TEST]]
    # recupera amostras de teste de acordo com a dobra em uso
    X_train, y_train = X_raw[idx_data[idx_dobra][TRAIN]], y_raw[idx_data[idx_dobra][TRAIN]]
    #verifica tamanho do conjunto de treino para indicar o numero de amostras disponiveis na dobra
    #print("tamanho de X_train - pool: ",X_train.shape)
    amostras_pool=X_train.shape
    n_amostras_pool = int(amostras_pool[0])
    #print("numero de amostras iniciais: ", n_amostras_pool)    
    # array de treinamento inicial 
    #initial_idx = np.random.choice(range(len(X_train)), size=1, replace=False)
    #inicial = X_train[initial_idx]
    #inicio aprendizado ativo
    for index in range(n_amostras_pool):
        train_idx = np.random.choice(range(len(X_train)))
        #print("posição da amostra recuperada: ",train_idx)
        #print("linha recuperada: ", X_train[train_idx].reshape(1, -1))
        atual = X_train[train_idx].reshape(1, -1)
        inicial = np.append(inicial, atual, axis=0)
        #print("ver se a base está crescendo: ",inicial.shape)
        X = deepcopy(inicial)
        #print(expected_error_reduction(learner, X))
        if expected_error_reduction(learner, X) > 0:
            #Ensina ao modelo ActiveLearner o registro solicitado (amostras vão para o topo).
            learner.teach(X=X_train[train_idx].reshape(1, -1),y=y_train[train_idx].reshape(1, ))
            new_score = learner.score(X_train, y_train)
            performance_history.append(new_score)
            #print ('Accuracy after query no. %d: %f' % (index+1, new_score))
            arquivo_performance.write('Accuracy after query no. %d: %f \n' % (index+1,new_score))
            predictions = learner.predict(X_test)
            #print ('Precision after query no. %d: %f' % (index+1, precision_score(y_test, predictions,average='macro',zero_division=1)))
            arquivo_performance.write('Precision after query no. %d: %f \n' % (index+1,precision_score(y_test, predictions,average='macro',zero_division=1)))
            #print ('F1 score after query no. %d: %f' % (index+1, f1_score(y_test, predictions,average='macro',zero_division=1)))
            arquivo_performance.write('F1 score after query no. %d: %f \n' % (index+1, f1_score(y_test, predictions,average='macro',zero_division=1)))
            #print ('Recall after query no. %d: %f' % (index+1, recall_score(y_test, predictions, average='macro',zero_division=1)))
            arquivo_performance.write('Recall after query no. %d: %f \n' % (index+1, recall_score(y_test, predictions, average='macro',zero_division=1)))
            #print ("========================================")
            arquivo_performance.write('======================================== \n')
        # apaga os modelos consultados
        X_train = np.delete(X_train, train_idx, axis=0)
        y_train = np.delete(y_train, train_idx)
        # diminui número de amostras 
        n_amostras_pool = n_amostras_pool -1
        #print("numero de amostras atuais",n_amostras_pool)
        for i in range(2):
            clear_output(wait=True)
            print(i,"Aguarde, em execução!", flush=True)
    
    np.savetxt(arquivo_history, performance_history,delimiter=",")
    arquivo_performance.close()     

import time
import sys
from datetime import datetime
from datetime import date

from sklearn.model_selection import StratifiedShuffleSplit, ShuffleSplit, train_test_split
from modAL.uncertainty import classifier_uncertainty
from modAL.models import ActiveLearner
from modAL.uncertainty import uncertainty_sampling
from modAL.expected_error import expected_error_reduction
from timeit import Timer
import time
import functools

#inicia relogio
t1 = time.time()
# Define o tamanho das divisões feitas no dataset (cross-validation)
n_splits = 10
# Define Tamanho inicial da amostra (toda estratégia parte de um tamanho mínimo aleatório).
init_size = 10
#define array de performance
performance_history = []
#define array de indices das partições
idx_data =[]
# cross validation bags - n_splits
data_cv = StratifiedShuffleSplit(n_splits= n_splits, random_state=0) 
data_cv.get_n_splits(X_raw, y_raw)
# chame a instância e gere os dados sobre a base original
type(data_cv.split(X_raw, y_raw))
# dividir os dados - A função split.split () retorna índices para amostras de treino e amostras de teste. 
# Ele examinará o número de validação cruzada especificado e retornará cada vez que treinar 
# e testar os índices de amostra usando os conjuntos de dados de treinamento e teste que podem 
# ser criados filtrando o conjunto de dados inteiro. Por exemplo idx_data[0][1], o primeiro indice faz referencia
# a dobra e o segundo indice faz referencia a posição da dobra (0 = treino e 1 = teste). Logo TRAIN=0 e TEST=1.
for train_index, test_index in data_cv.split(X_raw,y_raw):
    #print("TRAIN:", train_index, "TEST:", test_index)
    #print("n_split",n_splits,"TRAIN:", train_index, "TEST:", test_index)
    idx_data.append([train_index, test_index])
#verifica tamanho das dobras (numero de instâncias de cada dobra)
#print("tamanho de cada dobra: ",idx_data[3][0].shape)

TRAIN =0
TEST =1
#chama procedimento de aprendizado para todas as dobras
#classificadores=['knn']                                                             
classificadores=['knn','rf','nb','tree']
for classificador in classificadores:
    classifier = classificador
    for idx_dobra in range(n_splits):
        expected_error_reduction_function(X_raw, y_raw, idx_data, idx_dobra, TRAIN, TEST, classifier, init_size)

t2 = time.time()
time_elapsed = t2 -t1
print("tempo: ",time_elapsed)


1 Aguarde, em execução!


KeyboardInterrupt: 